In [1]:
! pip install pdfplumber pandas openpyxl

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.6 kB 222.6 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/43.6 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 306.0 kB/s eta 0:00:00
  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pillow-12.0.0-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
     ---------------------------------------- 0.0/67.8 kB ? eta -:--:--
     ------------------------------------ --- 61.4/67.8 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 67.8/67.8 kB 737.3 kB/s eta 0:00:00
  Using cached charset_normalizer-3.4.4-cp312-cp312-win_amd64.whl.metadata (38 kB)
  Using cached numpy-2.3.5-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-202


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pdfplumber
import pandas as pd
import re
import os

# ========= CONFIGURACIÓN ==========
CARPETA_PDF = "data/"          # carpeta donde tienes los PDF
SALIDA_EXCEL = "manifiestos.xlsx"
# ==================================


def limpiar_espacios(texto: str) -> str:
    """Colapsa espacios múltiples en uno solo."""
    return " ".join(texto.split())


def obtener_cabecera(pdf):
    """Lee solo la primera página para obtener empresa, dirección, título, reparto, vehículo y chofer."""
    page0 = pdf.pages[0]
    txt = page0.extract_text()
    lineas = [l.strip() for l in txt.split("\n") if l.strip()]

    empresa = lineas[0].strip()
    direccion = lineas[1].strip()

    titulo = ""
    for l in lineas:
        if "MANIFIESTO DE GUIAS" in l.upper():
            titulo = l.strip()
            break

    nro_reparto = vehiculo = chofer = ""
    for l in lineas:
        if "NRO REPARTO" in l and "VEHICULO" in l and "CHOFER" in l:
            cab = l
            m_rep = re.search(r"NRO REPARTO\s*([0-9]+)", cab)
            if m_rep:
                nro_reparto = m_rep.group(1)
            m_veh = re.search(r"VEHICULO\s*([A-Z0-9\-]+)", cab)
            if m_veh:
                vehiculo = m_veh.group(1)
            chofer = cab.split("CHOFER", 1)[1].strip()
            break

    return empresa, direccion, titulo, nro_reparto, vehiculo, chofer


def parsear_columna(texto_columna, base):
    """
    Parsea una columna (mitad de página) con 0, 1 o varios bloques de:
    NRO GUIA REMISIÓN   FECHA TRASLADO
    T001-00183406       28/11/2025
    DESTINATARIO
    72783876  NOMBRE...
    PUNTO DE ENTREGA
    MCDO. AZ PSTO 038   LA VICTORIA
    PESO
    132.05
    """
    registros = []
    lineas = [l.rstrip() for l in texto_columna.split("\n") if l.strip()]
    i = 0

    while i < len(lineas):
        linea = lineas[i].upper()

        if linea.startswith("NRO GUIA REMISIÓN"):
            # -------- línea con nro guía y fecha --------
            valores = lineas[i+1].strip()
            m_guia = re.search(r"(T\d{3}-\d+)", valores)
            m_fecha = re.search(r"(\d{2}/\d{2}/\d{4})", valores)

            nro_guia = m_guia.group(1) if m_guia else ""
            fecha_traslado = m_fecha.group(1) if m_fecha else ""

            # -------- DESTINATARIO --------
            # i+2 => "DESTINATARIO"
            dest_line = lineas[i+3].strip()
            dni = dest_line[:8]
            nombre = dest_line[8:].strip()
            dni = dni.strip()
            nombre = limpiar_espacios(nombre)

            # -------- PUNTO DE ENTREGA --------
            # i+4 => "PUNTO DE ENTREGA"
            punto_line = lineas[i+5].strip()
            punto_entrega = limpiar_espacios(punto_line)

            # -------- PESO --------
            # i+6 => "PESO"
            peso_line = lineas[i+7].strip()
            peso = peso_line.replace(",", ".").strip()

            reg = {
                "empresa": base["empresa"],
                "direccion": base["direccion"],
                "titulo": base["titulo"],
                "nro_reparto": base["nro_reparto"],
                "vehiculo": base["vehiculo"],
                "chofer": base["chofer"],
                "nro_guia": nro_guia,
                "fecha_traslado": fecha_traslado,
                "dni_destinatario": dni,
                "nombre_destinatario": nombre,
                "punto_entrega": punto_entrega,
                "peso": peso,
            }
            registros.append(reg)

            # saltamos este bloque (8 líneas)
            i += 8
            continue

        i += 1

    return registros


def extraer_datos_pdf(ruta_pdf):
    todos_registros = []

    with pdfplumber.open(ruta_pdf) as pdf:
        empresa, direccion, titulo, nro_reparto, vehiculo, chofer = obtener_cabecera(pdf)
        base = {
            "empresa": empresa,
            "direccion": direccion,
            "titulo": titulo,
            "nro_reparto": nro_reparto,
            "vehiculo": vehiculo,
            "chofer": chofer,
        }

        # Recorremos cada página y la cortamos en izquierda / derecha
        for page in pdf.pages:
            w, h = page.width, page.height
            mid = w / 2

            # Columna izquierda
            left = page.crop((0, 0, mid, h))
            txt_left = left.extract_text()
            if txt_left:
                todos_registros.extend(parsear_columna(txt_left, base))

            # Columna derecha
            right = page.crop((mid, 0, w, h))
            txt_right = right.extract_text()
            if txt_right:
                todos_registros.extend(parsear_columna(txt_right, base))

    return todos_registros


# ========= PROCESAR TODOS LOS PDF ==========
registros_totales = []

for archivo in os.listdir(CARPETA_PDF):
    if archivo.lower().endswith(".pdf"):
        ruta = os.path.join(CARPETA_PDF, archivo)
        print("Procesando:", archivo)
        registros_totales.extend(extraer_datos_pdf(ruta))

# Pasar a Excel
df = pd.DataFrame(registros_totales)
df.to_excel(SALIDA_EXCEL, index=False)
print("Archivo generado:", SALIDA_EXCEL)


Procesando: M2A-893 (28) 1.pdf
Procesando: M5O-842 (28) 1.pdf
Archivo generado: manifiestos.xlsx
